spreadsheet.ipynb

## Specifiche da implementare in Erlang
 Un worksheet deve essere una “matrice” di NxM celle ;
 Una cella può contenere qualsiasi tipo di dato primitivo o  il valore 'undef' .
 Il modulo del worksheet condiviso deve prevedere alcune funzioni di interfaccia sintatticamente definite
 1) new(name) -> spreadsheet | {error,reason} % Crea un nuovo foglio di nome “name” di dimensioni NxM di K tab (fogli)
  assegna il processo creatore come proprietario del foglio. I parametri N, M, K sono default nel modulo
 2) new(name, N, M, K) -> spreadsheet | {error,reason} % Crea un nuovo foglio di nome “name” di K fogli in cui ogni tab ha dimensioni NxM • assegna il processo creatore come proprietario del foglio

In [ ]:
-module(spreadsheet).
-export([new/1, new/4]).

-record(spreadsheet, {
    name,
    tabs = [],         % Lista di fogli (ogni foglio è una matrice NxM)
    owner = undefined  % Il proprietario del foglio
}).

% Funzione per creare un nuovo foglio con dimensioni default
new(Name) ->
    N = 10,  % Default N (numero di righe)
    M = 10,  % Default M (numero di colonne)
    K = 1,   % Default K (numero di tab)
    new(Name, N, M, K).

% Funzione per creare un nuovo foglio con dimensioni specificate
new(Name, N, M, K) when is_integer(N), is_integer(M), is_integer(K), N > 0, M > 0, K > 0 ->
    % Verifica se il nome del foglio esiste già
    case whereis(Name) of
        undefined ->
            % Creazione del processo proprietario
            Owner = self(),
            % Creazione dei tab (K fogli, ciascuno una matrice NxM)
            Tabs = lists:map(fun(_) -> create_tab(N, M) end, lists:seq(1, K)),
            % Creazione del record spreadsheet
            Spreadsheet = #spreadsheet{name = Name, tabs = Tabs, owner = Owner},
            % Registrazione del processo con il nome del foglio
            register(Name, self()),
            % Memorizzazione dello stato
            loop(Spreadsheet);
        _ ->
            {error, already_exists}
    end;
new(_, _, _, _) ->
    {error, invalid_parameters}.

% Funzione per creare un tab (una matrice NxM di celle)
create_tab(N, M) ->
    lists:map(fun(_) -> lists:duplicate(M, undef) end, lists:seq(1, N)).

% Loop principale del processo, dove si gestiscono i messaggi
loop(Spreadsheet) ->
    receive
        % Gestire messaggi per operazioni sul foglio di calcolo
        stop ->
            ok; % Fermare il processo
        _Other ->
            loop(Spreadsheet)
    end.


## Loop/1 e Loop/5(= diventerà starter/5)

**PROBLEMA**:se il pid del processo Spreadsheet coincide con self() per far sì che  coincida con il processo che lo ha generato (implementazione del requisito di proprietà), la shell resta in ascolto di messaggi senza fornire prompt.

**SOLUZIONE**:modello basato su processi separati, dove il processo che gestisce lo stato del foglio di calcolo (Spreadsheet) viene creato come un processo figlio separato dal processo proprietario.Proprietario: Il processo che crea e controlla il foglio di calcolo.
Processo separato per Spreadsheet: Il processo Spreadsheet deve essere separato dal proprietario usando la funzione spawn/1 o spawn/3 per creare un processo separato che gestisce il ciclo loop/1 del foglio di calcolo

In [ ]:
-module(spreadsheet).
-export([new/1, new/4, read_cell/3, write_cell/4]).

-record(spreadsheet, {
    name,
    tabs = [],
    owner = undefined,
    access_policies = []
}).

% Funzione per creare un nuovo foglio con dimensioni default (separato dal proprietario)
new(Name) ->
    N = 10,
    M = 10,
    K = 1,
    new(Name, N, M, K).

% Funzione per creare un nuovo foglio di calcolo con un processo separato
new(Name, N, M, K) when is_integer(N), is_integer(M), is_integer(K), N > 0, M > 0, K > 0 ->
    case whereis(Name) of
        undefined ->
            Owner = self(),  % Il proprietario è il processo che chiama new/4
            Pid = spawn(spreadsheet, loop, [Name, Owner, N, M, K]),  % Crea il processo separato
            register(Name, Pid),  % Registra il nome del processo per facilitarne l'accesso
            {ok, Pid};  % Restituisce il Pid del processo creato
        _ ->
            {error, already_exists}
    end;
new(_, _, _, _) ->
    {error, invalid_parameters}.

% Funzione per creare un tab (una matrice NxM di celle)
create_tab(N, M) ->
    lists:map(fun(_) -> lists:duplicate(M, undef) end, lists:seq(1, N)).

% Loop che gestisce il processo del foglio di calcolo
loop(Name, Owner, N, M, K) ->
    Tabs = lists:map(fun(_) -> create_tab(N, M) end, lists:seq(1, K)),
    Spreadsheet = #spreadsheet{name = Name, tabs = Tabs, owner = Owner},
    loop(Spreadsheet).

% Loop che gestisce lo Stato
loop(State = #spreadsheet{name = Name, tabs = Tabs, owner = Owner, access_policies = Policies}) ->
    receive


        stop ->
            ok;  % Termina il processo
        _Other ->
            loop(State)  % Continua a ricevere messaggi
    end.



Il processo Spreadsheet è ora separato dal processo chiamante, e si possono inviare messaggi da shell o da altri processi.
Il processo Spreadsheet rimane in esecuzione in background con uno Stato persistente,  ricevendo messaggi fino a quando non viene fermato esplicitamente con stop.
Il processo chiamante che ha creato il foglio è salvato come proprietario, e posso implementare controlli di accesso basati su questo.

## funzione loop/1 aggiornata,  con aggiunta delle funzioni di lettura e scrittura

In [ ]:
% loop che gestisce lo State del foglio di calcolo e le operazioni sui dati
loop(State = #spreadsheet{name = Name, tabs = Tabs, owner = Owner, access_policies = Policies}) ->
    io:format("Spreadsheet State loop started. Waiting for messages.~n"),
    receive

        % Handle get request
        %la Tab(la matrice) restituita da lists:nth(Tab, Tabs) è assegnata a TabMatrix
        {get, From, Tab, I, J} ->
            io:format("Received get request for Tab: ~p, Row: ~p, Col: ~p~n", [Tab, I, J]),
            if
                Tab > length(Tabs) orelse Tab < 1 ->
                    io:format("Tab index ~p is out of bounds~n", [Tab]),
                    From ! {cell_value, undef};
                true ->
                    TabMatrix = lists:nth(Tab, Tabs),
                    if
                        I > length(TabMatrix) orelse I < 1 ->
                            io:format("Row index ~p is out of bounds in Tab ~p~n", [I, Tab]),
                            From ! {cell_value, undef}; %If the indices are out of bounds, we return undef instead of trying to access an invalid position.
                        true ->
                            Row = lists:nth(I, TabMatrix),
                            if
                                J > length(Row) orelse J < 1 ->
                                    io:format("Col index ~p is out of bounds in Row ~p, Tab ~p~n", [J, I, Tab]),
                                    From ! {cell_value, undef};
                                true ->
                                    Value = lists:nth(J, Row),

                                    From ! {cell_value, Value}
                            end
                    end
            end,
            loop(State);

% Handle set request
        {set, From, Tab, I, J, Val} ->
            io:format("Received set request for Tab: ~p, Row: ~p, Col: ~p, Val: ~p~n", [Tab, I, J, Val]),
            if
                Tab > length(Tabs) orelse Tab < 1 ->
                    io:format("Tab index ~p is out of bounds~n", [Tab]),
                    From ! {set_result, false};
                true ->
                    TabMatrix = lists:nth(Tab, Tabs),
                    if
                        I > length(TabMatrix) orelse I < 1 ->
                            io:format("Row index ~p is out of bounds in Tab ~p~n", [I, Tab]),
                            From ! {set_result, false};
                        true ->
                            Row = lists:nth(I, TabMatrix),
                            if
                                J > length(Row) orelse J < 1 ->
                                    io:format("Col index ~p is out of bounds in Row ~p, Tab ~p~n", [J, I, Tab]),
                                    From ! {set_result, false};
                                true ->
                                    io:format("Setting value ~p at Tab: ~p, Row: ~p, Col: ~p~n", [Val, Tab, I, J]),
                                    NewRow = replace_nth(J, Val, Row),
                                    NewTabMatrix = replace_nth(I, NewRow, TabMatrix),
                                    NewTabs = replace_nth(Tab, NewTabMatrix, Tabs),
                                    NewState = State#spreadsheet{tabs = NewTabs},
                                    From ! {set_result, true},
                                    loop(NewState)
                            end
                    end
            end;





        {get_spreadsheetPid, From} ->

                io:format("Received request for spreadsheet state from: ~p~n", [From]),
                io:format("State to be returned: ~p~n", [State]),  % Stampa il valore di `State`
                % Invia il record #spreadsheet{} indietro
                From ! {spreadsheet_state, State},
                io:format("Sent spreadsheet state to: ~p~n", [From]),
                loop(State);  % Continua ad ascoltare messaggi

        {share, From, NewPolicies} ->
            if
                From =:= Owner ->
                    % Aggiornare le politiche di accesso
                    NewState = State#spreadsheet{access_policies = NewPolicies},
                    From ! {share_result, ok},
                    loop(NewState);
                true ->
                    % Se non è il proprietario, ritorna un errore
                    From ! {share_result,{error, not_owner}},
                    loop(State)
            end;
        {remove_policy, From, Proc} ->
            if
                From =:= Owner ->

                    NewPolicies = lists:filter(fun({P, _}) -> P =/= Proc end, Policies),
                    NewState = State#spreadsheet{access_policies = NewPolicies},
                    From ! {remove_policy_result, ok},
                    loop(NewState);

                true ->
                    From ! {error, not_owner},
                    loop(State)
            end;
        % Gestire messaggi per operazioni sul foglio di calcolo
        stop ->

            ok; % esce dal loop , ma attenzione non arresta Spreadsheet !!!
        _Other ->
            io:format("Unknown message received: ~p~n", [_Other]),
            loop(State)
    end.

To do
info(name) -> Spreadsheet_info Le informazioni devono contenere almeno:
• Numerodi celle x tab
• I permessi di lettura e scrittura

testare le politiche di accesso e inserire il controllo di accesso su get e set
Controllare i valori ritornati dalle funzioni delle specifiche (bool, ok,  ..)



# Persistenza del foglio di calcolo sotto forma di CSV
Soddisfatti i requisiti iniziali con new/1,new/4,share/2 per la creazione e la gestione delle politiche di accesso il progetto deve soddisfare ulteriori requisiti:
1) garantire la persistenza del foglio di calcolo con salvataggio dei record #spreadsheet sotto forma  di file CSV tenendo presente che le funzioni di interfaccia richieste devono essere
to_cvs(filename, spreadsheet) - > ok | {error, reason}
from_cvs(filename) - > speadsheet | {error, reason}.
In più bisogna tener presente che il foglio di calcolo accetta valori di ogni tipo più undef

In [ ]:
% Funzione per salvare il foglio di calcolo in un file CSV, accetta il registered name  spreadsheet
to_csv(Filename, SpreadsheetName) ->
    % Recupera il PID associato al nome registrato
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            % Invia un messaggio al processo per ottenere lo stato del foglio di calcolo
            Pid ! {get_spreadsheet, self()},
            receive
                {spreadsheet_state, #spreadsheet{name = Name, tabs = Tabs}} ->
                    case file:open(Filename, [write]) of
                        {ok, File} ->
                            % Scrivi il nome del foglio
                            io:format(File, "Spreadsheet Name: ~s~n", [Name]),
                            % Salva ogni tab come riga CSV
                            lists:foreach(fun(Tab) -> save_tab_to_csv(File, Tab) end, Tabs),
                            file:close(File),
                            ok;
                        {error, Reason} ->
                            {error, Reason}
                    end
            after 5000 ->
                {error, timeout}
            end
    end.

% Funzione per salvare ogni tab come riga CSV
save_tab_to_csv(File, Tab) ->
    lists:foreach(fun(Row) ->
        Line = lists:map(fun(Cell) -> format_cell(Cell) end, Row),
        io:format(File, "~s~n", [string:join(Line, ",")])
    end, Tab).

% Formattazione delle celle per CSV
format_cell(undef) -> "undef";
format_cell(Cell) -> io_lib:format("~p", [Cell]).


#### Persistenza dell' intero record con le già viste to_csv/2 e from_csv/2

**to_csv/2**
```console
...
 case file:open(Filename, [write]) of
        {ok, File} ->
            % Write the spreadsheet name
            io:format(File, "Spreadsheet Name: ~s~n", [Name]),

            % Write the owner information
            io:format(File, "Owner: ~p~n", [Owner]),

            % Write the access policies
            io:format(File, "Access Policies: ~p~n", [AccessPolicies]),

            % Write each tab as a row of CSV
            lists:foreach(fun(Tab) -> save_tab_to_csv(File, Tab) end, Tabs),
            file:close(File),
            ok;
        {error, Reason} ->
            {error, Reason}
    end.
```
The owner (which is the PID of the process that owns the spreadsheet) is serialized using ~p, so it’s written in a way that can be properly restored later.
The access policies list is serialized as a complete list of tuples (e.g., {proc1, read}), which can be parsed back when loading.





## From_csv/1

Binding Variables Correctly: Each field (like SpreadsheetName, Owner, and AccessPolicies) is bound inside its respective case expression, so it’s used only within the correct scope. This avoids the unsafe variable error.

PID and Access Policies Conversion:

The owner field is converted back to a PID using list_to_pid/1.
The access policies are converted back to a valid Erlang term using list_to_term/1 (assuming the access policies were written as a valid term using ~p in the to_csv/2 function).
Tabs Loading: After reading the metadata (name, owner, and access policies), the function proceeds to load the tabs using load_tabs_from_csv(File).

Error Handling: If any part of the file format doesn’t match the expected pattern (e.g., missing "Owner:" or "Access Policies:"), the function will log an error and return {error, invalid_format}.

Converting PIDs and Terms:
list_to_pid/1: Converts a string representation of a PID (like "<0.88.0>") back into an Erlang PID.

The erl_scan:string/1 function is used to tokenize the string that represents the Erlang term.
The . is added to complete the term since the string representation does not include this.
Parsing the Tokens:

Once tokenized, erl_parse:parse_term/1 converts the tokens into a valid Erlang term.
This step handles the conversion from the string representation of {proc1, read} and {proc2, write} into proper Erlang terms.

In [ ]:
from_csv(Filename) ->
    case file:open(Filename, [read]) of
        {ok, File} ->
            % Read the first line (Spreadsheet Name)
            SpreadsheetNameLine = io:get_line(File, ''),
            io:format("Raw Spreadsheet Name Line: ~p~n", [SpreadsheetNameLine]),

            % Extract the spreadsheet name
            case string:strip(SpreadsheetNameLine, both, $\n) of
                "Spreadsheet Name: " ++ SpreadsheetName ->
                    io:format("Extracted Spreadsheet Name: ~p~n", [SpreadsheetName]),

                    % Read the owner field
                    OwnerLine = io:get_line(File, ''),
                    io:format("Raw Owner Line: ~p~n", [OwnerLine]),

                    % Extract the owner
                    case string:strip(OwnerLine, both, $\n) of
                        "Owner: " ++ Owner ->
                            io:format("Extracted Owner PID: ~p~n", [Owner]),

                            % Read the access policies field
                            AccessPoliciesLine = io:get_line(File, ''),
                            io:format("Raw Access Policies Line: ~p~n", [AccessPoliciesLine]),

                            % Extract the access policies
                            case string:strip(AccessPoliciesLine, both, $\n) of
                                "Access Policies: " ++ AccessPoliciesString ->
                                    io:format("Extracted Access Policies (String): ~p~n", [AccessPoliciesString]),

                                    % Convert access policies string to a valid term
                                    case parse_term_from_string(AccessPoliciesString) of
                                        {ok, AccessPolicies} ->
                                            io:format("Parsed Access Policies: ~p~n", [AccessPolicies]),

                                            % Load the tabs from the remaining lines in the CSV
                                            Tabs = load_tabs_from_csv(File),
                                            io:format("Read Tabs: ~p~n", [Tabs]),  % Debug

                                            % Close the file
                                            file:close(File),

                                            % Construct the spreadsheet record
                                            Spreadsheet = #spreadsheet{
                                                name = list_to_atom(SpreadsheetName),  % Convert the name to an atom
                                                tabs = Tabs,
                                                owner = list_to_pid(Owner),  % Convert owner to a PID
                                                access_policies = AccessPolicies  % Use the parsed term for access policies
                                            },

                                            {ok, Spreadsheet};
                                        {error, Reason} ->
                                            io:format("Error parsing Access Policies: ~p~n", [Reason]),
                                            file:close(File),
                                            {error, invalid_access_policies}
                                    end;

                                _ ->
                                    io:format("Error: Invalid format for Access Policies line~n"),
                                    file:close(File),
                                    {error, invalid_format}
                            end;

                        _ ->
                            io:format("Error: Invalid format for Owner line~n"),
                            file:close(File),
                            {error, invalid_format}
                    end;

                _ ->
                    io:format("Error: Invalid format for Spreadsheet Name line~n"),
                    file:close(File),
                    {error, invalid_format}
            end;

        {error, Reason} ->
            io:format("Error opening file: ~p~n", [Reason]),
            {error, Reason}
    end.

% Funzione ausialiara per interpretare una stringa come termine
parse_term_from_string(String) ->
    % Tokenize the string
    case erl_scan:string(String ++ ".") of  % Add a period to complete the expression
        {ok, Tokens, _} ->
            % Parse the tokens into an Erlang term
            case erl_parse:parse_term(Tokens) of
                {ok, Term} -> {ok, Term};
                {error, Reason} -> {error, Reason}
            end;
        {error, Reason, _} -> {error, Reason}
    end.


## Timeout per la receive
```css

after 5000 ->
    {error, timeout}
end

```

To implement the functions for reading and writing values in the spreadsheet cells (get/5 and set/6) and their versions with timeouts, we can modify the spreadsheet process to handle these operations.
get/5: Retrieve the value at position (i, j) in tab k of the spreadsheet.
set/6: Write a value to the position (i, j) in tab k of the spreadsheet.

Both operations should be able to:
Access the specific tab.
Index into the matrix representing the tab to read or write values.
Support a version with timeout (get/6 and set/7).
We will implement these functions by interacting with the spreadsheet process and handling messages like {get, From, Tab, I, J} and {set, From, Tab, I, J, Val} in the spreadsheet's loop.

In [ ]:
% La funzione get/5 spedisce un messaggio al processo spreadsheet richiedendo il valore di una specifica cella del foglio di calcolo
get(Name, Tab, I, J) ->
    Pid = whereis(Name),  % Get the PID of the spreadsheet process
    Pid ! {get, self(), Tab, I, J},
    receive
        {cell_value, Value} -> Value
    after 5000 -> % Default timeout of 5 seconds
        timeout
    end.
% La funzione get/6 esegue lo stesso codice impostando un valore specifico per il timeout
get(Name, Tab, I, J, Timeout) ->
    Pid = whereis(Name),  % Get the PID of the spreadsheet process
    Pid ! {get, self(), Tab, I, J},
    receive
        {cell_value, Value} -> Value
    after Timeout ->
        timeout
    end.


%La funzione set/6 spedisce un messaggio al processo spreadsheet per aggiornare una cella specifica con un nuovo valore
set(Name, Tab, I, J, Val) ->
    Pid = whereis(Name),  % Get the PID of the spreadsheet process
    Pid ! {set, self(), Tab, I, J, Val},
    receive
        {set_result, Result} -> Result
    after 5000 -> % Default timeout of 5 seconds
        timeout
    end.

% set/7 specifica ulteriormente un valore di timeout desiderato
set(Name, Tab, I, J, Val, Timeout) ->
    Pid = whereis(Name),  % Get the PID of the spreadsheet process
    Pid ! {set, self(), Tab, I, J, Val},
    receive
        {set_result, Result} -> Result
    after Timeout ->
        timeout
    end.




# Aggiunta di specifiche di condivisione con politiche di accesso ai dati

share(spreadsheet, AccessPolicies) -> bool. Il proprietario del foglio può condidivere il foglio in Lettura o Scrittura con altri processi secondo AccessPolicies, che è una lista di {Proc,AP} dove:
 • Proc è un Pid/reg_name;
  • AP = read | write.
**Le policy di accesso ad un foglio possono cambiare in qualsiasi momento.**

Con la stessa share(spreadsheet, AccessPolicies) -> bool.
implemento  la gestione delle politiche di accesso (access policies) in modo da poter aggiornare solo le tuple {Proc, AP} presenti nella lista passata a share/2, mantenendo inalterate le altre

In breve:

Se un processo (Proc) è già presente nelle policy di accesso, il suo valore di accesso (AP) viene aggiornato con quello passato a share/2.
Se un processo (Proc) non è presente nella lista, viene aggiunto con il nuovo valore di accesso.
Le tuple {Proc, AP} non contenute nella lista passata a share/2 vengono mantenute inalterate.




In [ ]:

-module(spreadsheet).

%%.....stesso codice

% Funzione per condividere il foglio con policy di accesso
share(SpreadsheetName, AccessPolicies) when is_list(AccessPolicies) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {share, self(), AccessPolicies},  % Invia il nuovo messaggio per aggiornare le politiche
            receive
                {share_result, Result} -> Result
            end
    end.
    
% funzione ausiliarie di filtraggio per ottenere le Policy di Accesso aggiornate

    % List comprehension [Expression || Pattern <- List, Condition]
update_policies(NewPolicies, ExistingPolicies) ->
    % Step 1: Filter ExistingPolicies to exclude entries that are already in NewPolicies
    FilteredExistingPolicies = [
        Policy || 
        {Proc, _} = Policy <- ExistingPolicies, 
        not (
            lists:keymember(Proc, 1, NewPolicies) orelse
            is_pid(Proc) andalso lists:any(fun({NewProc, _}) -> whereis(NewProc) == Proc end, NewPolicies) orelse
            not is_pid(Proc) andalso lists:any(fun({NewProc, _}) -> NewProc == whereis(Proc) end, NewPolicies)
        )
    ],

    % Step 2: Return the combined list of NewPolicies and FilteredExistingPolicies
    FilteredExistingPolicies ++ NewPolicies.



% Loop principale del processo, dove si gestiscono i messaggi
loop(State = #spreadsheet{name = Name, tabs = Tabs, owner = Owner, access_policies = Policies}) ->
    receive
        {share, From, NewPolicies} ->
            if
                From =:= Owner ->
                    % Aggiornare le politiche di accesso usando update_policies/2
                    UpdatedPolicies = update_policies(NewPolicies, Policies),
                    NewState = State#spreadsheet{access_policies = UpdatedPolicies},
                    io:format("Updated access policies: ~p~n", [UpdatedPolicies]),  % Debug
                    From ! {share_result, true},
                    loop(NewState);
                true ->
                    % Se non è il proprietario, ritorna un errore
                    From ! {share_result, {error, not_owner}},
                    loop(State)
            end;
        stop ->
            ok; % Fermare il processo
        _Other ->
            loop(State)
    end.


I'm considering if we can merge in one list comprehension operation the act of not keeping in the merged list Newpolicies ++ Existing Policies  tuples that are in ExistingPolicies with {Proc, _} that is  lists:keymember(Proc, 1, NewPolicies) and tuples with {Proc, _} having Proc referring as a Pid o as a registered name to a member of Newpolicies with the first tuple"element referring to the same Process

In [ ]:
-module(upd_policies).
-export([update_policies/2]).
update_policies(NewPolicies, ExistingPolicies) ->
    % Step 1: Filter ExistingPolicies to exclude entries that are already in NewPolicies
    FilteredExistingPolicies = [
        Policy || 
        {Proc, _} = Policy <- ExistingPolicies, 
        not (
            lists:keymember(Proc, 1, NewPolicies) orelse
            is_pid(Proc) andalso lists:any(fun({NewProc, _}) -> whereis(NewProc) == Proc end, NewPolicies) orelse
            not is_pid(Proc) andalso lists:any(fun({NewProc, _}) -> NewProc == whereis(Proc) end, NewPolicies)
        )
    ],

    % Step 2: Return the combined list of NewPolicies and FilteredExistingPolicies
    FilteredExistingPolicies ++ NewPolicies.
    %With this single list comprehension, you can effectively:

    %Remove duplicates where the process in ExistingPolicies is already represented in NewPolicies (either as a PID or registered name).
    % Ensure that PIDs and registered names referring to the same process are handled correctly without introducing duplicates.


## implementazioni ulteriori sulla gestione delle politiche di accesso ai dati
2) Deve esserci un meccanismo per salvare le policy di accesso su disco e ricaricarle al riavvio del sistema (persistenza delle policy).
3) Una funzione deve permettere di rimuovere o modificare le policy di accesso esistenti


In [ ]:
% Funzione per rimuovere una policy di accesso per un processo specifico
remove_policy(SpreadsheetName, Proc) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {remove_policy, self(), Proc},
            receive
                {remove_policy_result, Result} -> Result
            end
    end.

% Funzione per salvare le politiche di accesso su disco
save_policies(SpreadsheetName) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {save_policies, self()},
            receive
                {save_policies_result, Result} -> Result
            end
    end.

% Funzione per caricare le politiche di accesso da disco
load_policies(SpreadsheetName) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {load_policies, self()},
            receive
                {load_policies_result, Result} -> Result
            end
    end.


## Le policy di accesso ai dati devono essere verificate nelle funzioni di lettura e scrittura

Per la lettura (get/4 e get/5), dobbiamo assicurarci che il processo chiamante abbia accesso in lettura.
Per la scrittura (set/5 e set/6), dobbiamo assicurarci che il processo chiamante abbia accesso in scrittura.
Introdurremo una funzione helper **check_access/3** per verificare se il processo chiamante ha i diritti di accesso corretti (lettura o scrittura). La funzione risolverà prima il processo chiamante e poi verificherà se il processo ha i diritti di accesso richiesti.

### Problemi affrontati
In un primo sviluppo check_access/3   only checks whether the PID (resolved from the registered name) is present in the access policies, but the original registered name (e.g., proc2) isn't being checked. As a result, if the registered name is in the policies but not the resolved PID, the function fails to find the matching policy, leading to {error, access_denied}. in un secondo sviluppo check_access/3 is currently checking the PID first, but the access policy only contains the registered name (proc2), not the PID (<0.98.0>). When the check_access/3 function is called with the PID, it doesn't find a matching policy for the PID in the list and returns {error, access_denied} even though the registered name has the appropriate access.


```console
check_access(PidOrName, Policies, RequiredAccess) ->
    % Resolve PidOrName to both PID and registered name if possible
    ResolvedPid = case is_pid(PidOrName) of
                      true -> PidOrName;  % If it's already a PID, keep it as the PID
                      false -> whereis(PidOrName)  % If it's a name, resolve to PID
                  end,

    % Find if the PID has a registered name
    RegisteredName = case is_pid(PidOrName) of
                         true -> find_registered_name(ResolvedPid);  % Get registered name for the PID
                         false -> PidOrName  % If it's already a name, keep it
                     end,

    % Check both the resolved PID and the registered name in the access policies
    case lists:keyfind(ResolvedPid, 1, Policies) of
        {ResolvedPid, Access} when Access == RequiredAccess -> ok;
        _ ->
            case lists:keyfind(RegisteredName, 1, Policies) of
                {RegisteredName, Access} when Access == RequiredAccess -> ok;
                _ -> {error, access_denied}
            end
    end.

% Helper function to find the registered name of a process by its PID
find_registered_name(Pid) ->
    lists:foldl(
        fun(Name, Acc) ->
            case whereis(Name) of
                Pid when Pid =/= undefined -> Name;  % Return the name if it matches the PID
                _ -> Acc  % Otherwise, keep searching
            end
        end,
        undefined,
        registered()
    ).

```


% Modifiche alle funzioni get/4 e get/5
```console
get(SpreadsheetName, TabIndex, I, J) ->
    get(SpreadsheetName, TabIndex, I, J, infinity). %infiniti riconduce get/4 a get/5

get(SpreadsheetName, TabIndex, I, J, Timeout) ->
    case whereis(SpreadsheetName) of
        undefined -> {error, spreadsheet_not_found};
        Pid ->
            % Check if the caller has read access
            %Prima di inviare la richiesta get al processo del foglio di calcolo, controlliamo se il processo %chiamante (self()) ha accesso in lettura. In caso contrario, restituiamo {error, access_denied}.
            case check_access(self(), Spreadsheet#spreadsheet.access_policies, read) of
                ok ->
                    Pid ! {get, self(), TabIndex, I, J},
                    receive
                        {get_result, Value} -> Value
                    after Timeout -> {error, timeout}
                    end;
                {error, access_denied} -> {error, access_denied}
            end
    end.
```

% Modifiche alle funzioni set/5 e set/6
```console
set(SpreadsheetName, TabIndex, I, J, Value) ->
    set(SpreadsheetName, TabIndex, I, J, Value, infinity).

set(SpreadsheetName, TabIndex, I, J, Value, Timeout) ->
    case whereis(SpreadsheetName) of
        undefined -> {error, spreadsheet_not_found};
        Pid ->
            % Check if the caller has write access
            %Prima di inviare la richiesta di set al processo del foglio di calcolo, controlliamo se il processo %chiamante (self()) ha accesso in scrittura. In caso contrario, restituiamo {error, access_denied}.
            case check_access(self(), Spreadsheet#spreadsheet.access_policies, write) of
                ok ->
                    Pid ! {set, self(), TabIndex, I, J, Value},
                    receive
                        {set_result, Result} -> Result
                    after Timeout -> {error, timeout}
                    end;
                {error, access_denied} -> {error, access_denied}
            end
    end.

```

## Aggiunta di funzioni per la persistenza delle policy di accesso

In [ ]:
-module(spreadsheet).
-export([new/1, new/4, share/2, remove_policy/2, save_policies/1, load_policies/1]).

%%.............

% Funzione per condividere il foglio con policy di accesso
share(SpreadsheetName, AccessPolicies) when is_list(AccessPolicies) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {share, self(), AccessPolicies},
            receive
                {share_result, Result} -> Result
            end
    end.

% Funzione per rimuovere una policy di accesso per un processo specifico
remove_policy(SpreadsheetName, Proc) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {remove_policy, self(), Proc},
            receive
                {remove_policy_result, Result} -> Result
            end
    end.
% Funzione per salvare le politiche di accesso su disco
{save_policies, From} ->
            % Codice per salvare le politiche su disco (implementazione seguente)
            % Serializzare le policy e salvarle in un file
            {ok, File} = file:open(State#spreadsheet.name ++ "_policies.dat", [write]),
            ok = io:format(File, "~p.", [Policies]),
            file:close(File),
            From ! {save_policies_result, ok},
            loop(State);


% Funzione per caricare le politiche di accesso da disco
load_policies(SpreadsheetName) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {load_policies, self()},
            receive
                {load_policies_result, Result} -> Result
            end
    end.

% Loop che gestisce il processo del foglio di calcolo
loop(Name, Owner, N, M, K) ->
    Tabs = lists:map(fun(_) -> create_tab(N, M) end, lists:seq(1, K)),
    Spreadsheet = #spreadsheet{name = Name, tabs = Tabs, owner = Owner},
    loop(Spreadsheet).

% Loop
loop(State = #spreadsheet{owner = Owner, access_policies = Policies}) ->
    receive
        {share, From, AccessPolicies} ->
            if
                From =:= Owner ->
                    NewState = State#spreadsheet{access_policies = AccessPolicies},
                    From ! {share_result, ok},
                    loop(NewState);
                true ->
                    From ! {share_result, {error, not_owner}},
                    loop(State)
            end;

        {remove_policy, From, Proc} ->
            if
                From =:= Owner ->
                    NewPolicies = lists:filter(fun({P, _}) -> P =/= Proc end, Policies),
                    NewState = State#spreadsheet{access_policies = NewPolicies},
                    From ! {remove_policy_result, ok},
                    loop(NewState);
                true ->
                    From ! {remove_policy_result, {error, not_owner}},
                    loop(State)
            end;





        {save_policies, From} ->
            % Codice per salvare le politiche su disco (implementazione seguente)
            % Serializzare le policy e salvarle in un file
            {ok, File} = file:open(State#spreadsheet.name ++ "_policies.dat", [write]),
            ok = io:format(File, "~p.", [Policies]),
            file:close(File),
            From ! {save_policies_result, ok},
            loop(State);

        {load_policies, From} ->
            % Codice per caricare le politiche da disco (implementazione seguente)
            {ok, File} = file:open(State#spreadsheet.name ++ "_policies.dat", [read]),
            {ok, Policies} = io:fread(File, "~p"),
            file:close(File),
            NewState = State#spreadsheet{access_policies = Policies},
            From ! {load_policies_result, ok},
            loop(NewState);

        stop -> ok;
        _Other -> loop(State)
    end.




# Gestione del possesso della proprietà del processo spreadsheet
To handle the situation where the Erlang shell crashes and you lose ownership of the spreadsheet process (because the owner is tied to the original shell's PID), you can implement a mechanism that allows the owner to be reassigned to a new process or PID. This is especially important in a fault-tolerant system where processes can crash, and we need to recover gracefully.


Allow Reassigning Ownership: 
Implement a new function (e.g., reassign_owner/2) that allows changing the #spreadsheet.owner field to a new PID (the current shell's PID).
Ownership Check: Ensure that only the current owner or an administrator/special role can trigger this reassignment.
Error Handling: Handle errors gracefully if an unauthorized process tries to reassign ownership.


In [ ]:
% Funzione che permette di riassegnare  il proprietario  di spreadsheet se il processo chiamamante è il corrente proprietario
reassign_owner(SpreadsheetName, NewOwnerPid) when is_pid(NewOwnerPid) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found}; % The spreadsheet process doesn't exist
        Pid ->
            Pid ! {reassign_owner, self(), NewOwnerPid},
            receive
                {reassign_owner_result, Result} -> Result
            end
    end.

% modifiche a loop/1
loop(State = #spreadsheet{name = Name, owner = Owner} = SpreadsheetState) ->
    receive
        % Handle ownership reassignment
        {reassign_owner, From, NewOwner} ->
            if
                From =:= Owner ->  % Only the current owner can reassign ownership
                    NewState = State#spreadsheet{owner = NewOwner},
                    io:format("Ownership reassigned from ~p to ~p~n", [Owner, NewOwner]),
                    From ! {reassign_owner_result, ok},
                    loop(NewState);
                true ->
                    From ! {reassign_owner_result, {error, unauthorized}},
                    loop(State)
            end;

        % Existing handlers...

        % Stop the spreadsheet process
        stop ->
            ok;
            
        _Other ->
            io:format("Unknown message received: ~p~n", [_Other]),
            loop(State)
    end.


# implementare la funzione info/1, che fornirà informazioni dettagliate sul foglio di calcolo, tra cui:

**Numero di celle per scheda** (lo calcoleremo contando il numero di righe e colonne in ogni scheda e moltiplicando i valori).
**Autorizzazioni di lettura e scrittura**: forniremo una panoramica dei processi che hanno accesso in lettura e scrittura in base al campo access_policies.

**Numero totale di schede**: quante schede (fogli di lavoro) ha il foglio di calcolo.
**Proprietario del foglio di calcolo**: visualizza il proprietario del foglio di calcolo.
**Numero totale di celle**: somma il numero totale di celle in tutte le schede.
Includi la **data dell'ultima modifica** dei dati del foglio di calcolo

La funzione prende il nome (o PID) del foglio di calcolo come input.
Interroga il processo del foglio di calcolo per conoscere il suo stato (tramite un messaggio), da cui estrarrà le informazioni richieste e le restituirà in modo strutturato.

In [ ]:
-record(spreadsheet, {
    name,
    tabs = [],                 % List of tabs (each tab is a matrix NxM)
    owner = undefined,          % Owner of the spreadsheet
    access_policies = [],       % List of {Proc, Access} tuples
    last_modified = undefined   % Timestamp of the last modification
}).

% Function to get spreadsheet information
info(SpreadsheetName) ->
    case whereis(SpreadsheetName) of
        undefined -> {error, spreadsheet_not_found};  % If the process is not found
        Pid ->
            % Send a request to the spreadsheet process to get its state
            Pid ! {get_info, self()},
            receive
                {info_result, Info} -> {ok, Info};
                {error, Reason} -> {error, Reason}
            after 5000 -> {error, timeout}
            end
    end.
loop(State = #spreadsheet{name = Name, tabs = Tabs, owner = Owner, access_policies = AccessPolicies}) ->
    receive
        {get_info, From} ->
            % Calculate the number of cells and total tabs
            TotalTabs = length(Tabs),
            TotalCells = lists:sum([length(Tab) * length(lists:nth(1, Tab)) || Tab <- Tabs]),
            
            % Split access policies into read and write permissions
            ReadPermissions = [Proc || {Proc, read} <- AccessPolicies],
            WritePermissions = [Proc || {Proc, write} <- AccessPolicies],

            % Create the info result
            Info = #{name => Name,
                     owner => Owner,
                     total_tabs => TotalTabs,
                     total_cells => TotalCells,
                     read_permissions => ReadPermissions,
                     write_permissions => WritePermissions},

            % Send the result back to the requester
            From ! {info_result, Info},
            loop(State);

        % Other cases...

        

        _Other ->
            io:format("Unknown message received: ~p~n", [_Other]),
            loop(State)
    end.


In [ ]:
% set/5 function (with infinite timeout)
set(SpreadsheetName, TabIndex, I, J, Value) ->
    set(SpreadsheetName, TabIndex, I, J, Value, infinity).

% set/6 function (with custom timeout)
set(SpreadsheetName, TabIndex, I, J, Value, Timeout) ->
    case whereis(SpreadsheetName) of
        undefined -> {error, spreadsheet_not_found};
        Pid ->
            % Send the set request to the spreadsheet process
            Pid ! {set, self(), TabIndex, I, J, Value},
            receive
                {set_result, ok} -> ok;
                {error, access_denied} -> {error, access_denied}
            after Timeout ->
                {error, timeout}
            end
    end.


In [ ]:
to_csv e from_csv ...

## Here comes the approach to distributed version

### Overview of **gen_server**
gen_server is a generic server behavior that abstracts the common functionalities of a server process in Erlang/OTP. It provides a framework for implementing servers, handling everything from message passing and event handling to state management and concurrency. By using gen_server, *developers can focus on the business logic specific to their application while relying on the proven, robust infrastructure provided by OTP for handling process life cycles, communication, and error recovery.*
Components of gen_server
gen_server consists of several key components:

Callbacks: You implement specific callback functions that gen_server will call during its operation. These include:

init/1: Initializes the server. This is called when the server starts.
handle_call/3: Handles synchronous calls to the server.
handle_cast/2: Handles asynchronous messages to the server.
handle_info/2: Handles all other messages that don't fit into the above categories.
terminate/2: Cleans up when the server is shutting down.
code_change/3: Handles dynamic code upgrades.
API Functions: These are functions you define to interact with the server. They usually wrap gen_server calls like gen_server:call() and gen_server:cast().

### Applying gen_server to a Distributed Spreadsheet
When designing a distributed spreadsheet with Erlang, using gen_server is indeed appropriate because it provides robust process management and simplifies handling of state and communication. The gen_server framework can efficiently manage the complexities associated with a distributed environment, such as concurrency, fault tolerance, and state consistency.

Key Considerations for a Distributed Spreadsheet:
**State Management**: The spreadsheet can be represented as a record in Erlang, which includes not just the data but also metadata such as access policies and possibly a revision history to manage changes over time. The state of the gen_server would encapsulate this record.
C**oncurrency and Access Control**: Handling concurrent access to different parts of the spreadsheet requires careful management of state to ensure data integrity and consistency. gen_server's sequential handling of requests can naturally serialize access to data, but for finer-grained concurrency control (e.g., allowing simultaneous edits in different tabs), additional mechanisms might be needed.
Distributed Operations: Since the spreadsheet is distributed, operations may need to span multiple nodes. The gen_server can handle local operations, and for operations that need consistency across nodes, you might use distributed Erlang features or a distributed database like Mnesia.
**Node Failure and Recovery**: One of the strengths of using gen_server within the OTP framework is its built-in support for handling failures through supervisors. You can design your system to automatically restart failed spreadsheet processes, possibly restoring their last known good state from a backup or log.

